In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import math
from tqdm import tqdm

In [2]:
data_path='raw_data'
fs = os.listdir(data_path)  #raw_data 文件夹下的所有文件列表
len(fs)

4500

In [3]:
data=pd.DataFrame() 
for f in tqdm(fs):
    file_path = os.path.join(data_path, f) 
    df = pd.read_csv(file_path, usecols=[0,1,2],names = ["Ax", "Ay", "Az"] )
    if ('D01' in f) or ('D02' in f) or('D03' in f) or('D04' in f):
        df=df[:10000]
    #降采样:200Hz --> 100Hz
    df=df.loc[::2] #选取df1的奇数行
    

    Sensor1_Resolution = 13
    Sensor1_Range = 16
    g_S1 = (2*Sensor1_Range/2**Sensor1_Resolution)

    df['Ax']=g_S1*df['Ax']
    df['Ay']=g_S1*df['Ay']
    df['Az']=g_S1*df['Az']

    A=[]
    for i in range(df.shape[0]):
        A.append( np.sqrt(df.iloc[i]['Ax']**2+df.iloc[i]['Ay']**2+df.iloc[i]['Az']**2) )
    df['SVM']=A
    
    A=[]
    for i in range(df.shape[0]):
        A.append( np.sqrt(df.iloc[i]['Ay']**2+df.iloc[i]['Az']**2) )
    df['HOR']=A  #水平合加速度？
    
    #特征3 VER=np.sqrt(Ax^2+Az^2)
    A=[]
    for i in range(df.shape[0]):
        A.append( np.sqrt(df.iloc[i]['Ax']**2+df.iloc[i]['Az']**2) )
    df['VER']=A  #竖直合加速度？

    
    max_N = df['SVM'].max()
    max_N_index = df.index[df.SVM == max_N][0]  
    #最大的 SVM值对应的 idex可能是一个列表，取[0]表示第一次出现最大值的index
    min_N = df.SVM.min()
    min_N_index = df.index[df.SVM == min_N][0]
    len_df = len(df)  # df.shape[0]

    # We work with a window of 300 measurements (equivalent to 3 seconds of activity recording)
    #分三种情况，取300个数据(如果有的话)，因为频率为100Hz，3s 时间 有300个数据
    if (max_N_index - 150<0):
        df = df[0: 301]
        
    else:
        if (max_N_index + 150+1> len_df):
            df = df[len_df-302:len_df-1]
            
        else:
            # extract the central window
            df = df[max_N_index - 150: max_N_index + 150+1]
            
    #重置索引
    df.index=list(range(df.shape[0]))

    for c in ['Ax','Ay','Az','SVM','VER','HOR']:
        df['min_'+c]=df[c].min()
        df['max_'+c]=df[c].max()
        df['mean_'+c]=df[c].mean()
        df['median_'+c]=df[c].median()
        df['var_'+c]=df[c].var()
        df['std_'+c]=df[c].std()
        df['kurtosis_'+c]=df[c].kurtosis()
        df['skewness_'+c]=df[c].skew()
        df['range_'+c]=df[c].max()- df[c].min()
    
    df['corr_valueXY']=df["Ax"].corr(df["Ay"])
    df['corr_valueXZ']=df["Ax"].corr(df["Az"])
    df['corr_valueYZ']=df["Ay"].corr(df["Az"])
    
    df['corr_SVM_VER']=df["SVM"].corr(df["VER"])
    df['corr_SVM_HOR']=df["SVM"].corr(df["HOR"])
    df['corr_HOR_VER']=df["HOR"].corr(df["VER"])

    pitch=[]
    for i in range(df.shape[0]):
        pitch.append(math.atan2(-df['Ax'].loc[i],df['Az'].loc[i]))  
        #弧度制表示，这样对于12维特征，不需要特征缩放
    df['pitch']=pitch
    
    df['max_pitch']=df["pitch"].max()
    df['min_pitch']=df["pitch"].min()
    df['mean_pitch']=df["pitch"].mean()
    df['median_pitch']=df["pitch"].median()
    df['var_pitch']=df["pitch"].var()
    df['std_pitch']=df["pitch"].std()
    
    
    df['label']=f[0:1]
    df['label']=df['label'].map({'D':0,'F':1 })
    
    df.drop(['Ax','Ay','Az','SVM','HOR','VER'],axis=1,inplace= True)
    df=df[0:1]
    
    data=pd.concat([data,df],axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 4500/4500 [3:37:34<00:00,  2.90s/it]


In [4]:
data

,min_Ax,max_Ax,mean_Ax,median_Ax,var_Ax,std_Ax,kurtosis_Ax,skewness_Ax,range_Ax,min_Ay,...,corr_SVM_HOR,corr_HOR_VER,pitch,max_pitch,min_pitch,mean_pitch,median_pitch,var_pitch,std_pitch,label
0,-0.390625,0.371094,0.013756,0.007812,0.015765,0.125558,0.854790,0.290477,0.761719,-1.703125,...,0.997945,0.102362,-2.258941,3.130420,-3.141593,-0.205753,-0.739975,5.159133,2.271373,0
0,-0.527344,0.640625,0.013367,0.003906,0.028548,0.168962,2.077816,0.396032,1.167969,-1.539062,...,0.996858,0.499423,2.833540,3.132897,-3.141593,-0.161104,-0.194241,3.376536,1.837535,0
0,-0.539062,0.625000,0.096190,0.105469,0.043234,0.207927,-0.017689,-0.177633,1.164062,-1.464844,...,0.987008,0.282821,2.612003,3.133900,-3.141593,-0.840431,-2.324626,6.245205,2.499041,0
0,-0.316406,0.480469,0.085574,0.105469,0.021131,0.145366,-0.232488,-0.241202,0.796875,-1.417969,...,0.995986,0.240237,-2.622447,3.128935,-3.141593,-1.153116,-2.518684,6.157227,2.481376,0
0,-0.523438,0.578125,0.018805,0.019531,0.027538,0.165946,1.438329,0.119484,1.101562,-1.578125,...,0.992214,0.351024,-1.805519,3.127509,-3.141593,-0.232894,-0.540420,2.743725,1.656419,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-0.832031,-0.746094,-0.789374,-0.789062,0.000080,0.008959,3.422871,0.115047,0.085938,-0.312500,...,0.473877,0.361525,1.010436,1.054871,0.971112,1.006320,1.005647,0.000141,0.011865,1
0,-0.746094,-0.691406,-0.714000,-0.714844,0.000087,0.009312,-0.118073,0.056823,0.054688,-0.359375,...,0.558312,0.363358,0.888066,0.931497,0.851566,0.889849,0.890119,0.000167,0.012937,1
0,-0.824219,-0.761719,-0.781821,-0.781250,0.000068,0.008242,2.248672,-0.599216,0.062500,-0.359375,...,0.593155,0.459771,1.018867,1.052728,0.979252,1.012165,1.011754,0.000124,0.011147,1
0,-0.867188,-0.820312,-0.845372,-0.847656,0.000057,0.007565,0.511361,0.001038,0.046875,-0.292969,...,0.454293,0.311709,1.096200,1.147942,1.073766,1.109512,1.109011,0.000121,0.011015,1


In [5]:
data.to_csv('sisfall_3s_rad_pitch_100Hz.csv',index=False) #共提取出了4500个文件